# Phân loại lỗi bề mặt sản phẩm công nghiệp sử dụng mô hình "bag of word"

Tổng quan: Ở bài tập này chúng ta sẽ lần lượt thực hành các bước để xây dựng một hệ thống học máy cho bài toán phân loại lỗi bề mặt sản phẩm công nghiệp: trích xuất đặc trưng, xây dựng mô hình phân loại, phân loại trên hình ảnh thực tế. Bài tập yêu cầu các kiến thức về lập trình Python với các thư viện: OpenCV, scikit-learn, pickle, numpy.

Trong trường hợp chưa cài thành công opencv contrib, chạy các lệnh sau trên cửa sổ terminal:
pip uninstall opencv-contrib-python
pip uninstall opencv-python
pip install opencv-python==3.4.0.12
pip install opencv-contrib-python==3.4.0.12
pip uninstall numpy
pip install numpy==1.15.2

##  Kiểm tra thư viện openCV


In [10]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [11]:
cd /content/drive/'My Drive'/Code/BOW

[Errno 2] No such file or directory: '/content/drive/My Drive/Code/BOW'
/content


## Khai báo thư viện

In [12]:
!pip install opencv-contrib-python==4.4.0.44
import warnings
warnings.filterwarnings("ignore")
import os
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt
import pickle
import sklearn

from sklearn.cluster import KMeans
from scipy.spatial.distance import cdist

## Đọc dữ liệu ảnh từ thư mục

In [13]:
dataDir='./xu_ly_anh/'
classes=['L1', 'L2', 'L3', 'L4', 'L5', 'L6']

In [14]:
# Định nghĩa hàm load dữ liệu 
def LoadData(dataDir,new_size=None):
    if not new_size is None:
        img_rows, img_cols = new_size
    classes=[]
    for _,dirs,_ in os.walk(dataDir):
        classes=dirs # Số lớp chính bằng số thư mục con
        break  
    num_classes=len(classes)    
    ValidPercent=30  # 30% số ảnh trong 1 lớp sẽ dùng cho kiểm thử còn lại để huấn luyện
    X_tr=[]
    Y_tr=[]
    X_te=[]
    Y_te=[]    
    for idx,cl in enumerate(classes):
        for _,_,files in os.walk(dataDir+cl+'/'):               
            l=len(files)
            for f in files:
                # Lấy ngẫu nhiên 1 số từ 0 đến 100 để quyết định dùng mẫu này để train hay test
                r=np.random.randint(100) 
                img_path=dataDir+cl+'/'+f
                img=cv.imread(img_path)
                img = cv.cvtColor(img, cv.COLOR_BGR2RGB)
                if not new_size is None:
                    img=cv.resize(img,(img_rows,img_cols))
                if (r>ValidPercent):
                    X_tr.append(img)
                    Y_tr.append(idx)  
                else:
                    X_te.append(img)
                    Y_te.append(idx)                      
    return X_tr, Y_tr, X_te, Y_te

In [16]:
new_size=(256,256)
img_train,label_train, img_test, label_test=LoadData(dataDir,None)
"""
Some information of the data
"""
print("img for train: %d (Sử dụng tạo từ điển và khâu huấn luyện)" % (len(img_train)))
print("label for train: %d" % (len(label_train)))

print("img for test: %d" % (len(img_test)))
print("label for test: %d" % (len(label_test)))

# Hiển thị ngẫu nhiên 12 ảnh mẫu
idxs=np.random.permutation(len(img_train))
plt.figure(figsize = (10,10))
for i in range(12):  # Lấy ngẫu nhiên 12 mẫu trong tập train
    idx = idxs[i]
    plt.subplot(4,3,i+1)
    plt.imshow(img_train[idx])
    plt.title(classes[label_train[idx]])
plt.show()

img for train: 0 (Sử dụng tạo từ điển và khâu huấn luyện)
label for train: 0
img for test: 0
label for test: 0


IndexError: ignored

<Figure size 720x720 with 0 Axes>

## Xây dựng đặc trưng

Ở đây chúng ta sẽ lần lượt thực hiện các công việc: trích xuất đặc trưng SIFT (SIFT keypoints descriptor), xây dựng từ điển và mô hình Bag-of-Word (BoW). Chỉ sử dụng tập huấn luyện 

### Trích xuất đặc trưng SIFT hoặc SURF (cục bộ)

Hàm extract_sift_features() nhận đầu vào là list ảnh ta đã xây dựng được từ câu hỏi trước và trả về list image_descriptors, phần tử thứ p trong list image_descriptors lại là một list chứa các SIFT keypoints descriptor ứng với ảnh thứ p trong list X.

Để khởi tạo đối tượng SIFT trong OpenCV ta sử dụng lệnh: sift = cv2.xfeatures2d.SIFT_create(). Đối tượng này có phương thức detectAndCompute trả về 2 outputs kp và des, kp là một list chứa các keypoints được detect bởi SIFT, des là một numpy array chứa len(kp) vectors 128 chiều. Chúng ta sẽ dùng các des này để phục vụ bài toán phân loại.

In [ ]:
def extract_sift_features(X):
    image_descriptors = []
    sift = cv.xfeatures2d.SIFT_create()
    for i in range(len(X)):
        kp, des = sift.detectAndCompute(X[i], None)         
        image_descriptors.append(des)
    return image_descriptors

In [ ]:
# Nếu opencv không có đủ thư viện: AttributeError: 'module' object has no attribute 'xfeatures2d'
image_descriptors = extract_sift_features(img_train)


NameError: ignored

In [ ]:
print(len(image_descriptors))
print(type(image_descriptors[0][0]))
print((image_descriptors[0].shape))
for i in range(10):
    print('Image {} has {} descriptors'.format(i, len(image_descriptors[i])))

### Xây dựng từ điển
hàm kmeans_bow() nhận đầu vào là một list gồm tất cả các descriptors của các ảnh trong tập X và số cụm num_clusters, sử dụng thuật toán KMeans trong scikit-learn phân cụm các vector descriptors này thành num_clusters cụm. Hàm trả về một danh sách center của các cụm. Vì thao tác này mất rất nhiều thời gian nên ta sẽ lưu danh sách trả về bởi hàm kmeans_bow() ra một file nhị phân (sử dụng thư viện pickle) để tiện sử dụng sau này.
Trước tiên ta sẽ xây dựng một list chứa tất cả các descriptors:

In [ ]:
all_descriptors = []
for descriptors in image_descriptors:
    if descriptors is not None:
        for des in descriptors:
            all_descriptors.append(des)
print('Total number of descriptors: %d' %(len(all_descriptors)))

Định nghĩa hàm kmeans_bow().

Gợi ý: Trong thư viện sklearn, để khởi tạo một đối tượng kmeans ta sử dụng lệnh:
kmeans = sklearn.cluster.KMeans(n_clusters=num_clusters)

Đối tượng kmeans có phương thức fit, nhận vào các điểm dữ liệu (trong bài toán hiện tại của chúng ta là list all_descriptors) và tìm ra n_clusters vector là tâm của các cụm dữ liệu. Để lấy được các vector này ta sử dụng thuộc tính $cluster_centers_$  của đối tượng kmeans

In [ ]:
def kmeans_bow(all_descriptors, num_clusters):
    bow_dict = []
    kmeans = KMeans(n_clusters=num_clusters).fit(all_descriptors)
    bow_dict = kmeans.cluster_centers_
    return bow_dict

In [ ]:
num_clusters = 96
if not os.path.isfile('bow_dictionary96.pkl'):
    BoW = kmeans_bow(all_descriptors, num_clusters)    
    pickle.dump(BoW, open('bow_dictionary96.pkl', 'wb'))
else:
    BoW = pickle.load(open('bow_dictionary96.pkl', 'rb'))
print(len(BoW))
print(len(BoW[0]))

### Xây dựng vector đặc trưng với mô hình BoW đã thu được

Xây dựng hàm create_features_bow() nhận đầu vào là list image_descriptors, list BoW và num_clusters ở trên, trả về list X_features, trong đó phần tử thứ p của X_vectors là vector đặc trưng theo mô hình BoW ứng với ảnh thứ p, tập keypoint descriptors thứ p. Hãy chú ý sự tương ứng các phần tử trong 4 danh sách: X_tr, Y_tr, image_descriptors, X_features.

In [ ]:
from scipy.spatial.distance import cdist
def create_features_bow(image_descriptors, BoW, num_clusters):
    X_features = []
    for i in range(len(image_descriptors)):
        features = np.array([0] * num_clusters) # <=> features=np.zeros(num_clusters,dtype=int)        
        if image_descriptors[i] is not None:
            distance = cdist(image_descriptors[i], BoW)            
            argmin = np.argmin(distance, axis=1)            
            for j in argmin:
                features[j] += 1
        X_features.append(features)
    return X_features


#### Tạo tập huấn luyện dựa vào từ điển

In [ ]:
image_descriptors = extract_sift_features(img_train)
X_tr = create_features_bow(image_descriptors, BoW, num_clusters)
X_tr=np.array(X_tr)
Y_tr=np.array(label_train)
print('Dữ liệu huấn luyện ')
print("train data: " + str(X_tr.shape))
print("train label: " + str(Y_tr.shape))

#### Tạo tập kiểm thử dựa vào từ điển

In [ ]:
image_descriptors = extract_sift_features(img_test)
X_te = create_features_bow(image_descriptors, BoW, num_clusters)
X_te=np.array(X_te)
Y_te=np.array(label_test)
print('Dữ liệu kiểm thử ')
print("train data: " + str(X_te.shape))
print("train label: " + str(Y_te.shape))

NameError: ignored

###  Xây dựng mô hình phân lớp dựa vào dữ liệu mới tạo
Chúng ta đã xây dựng được vector đặc trưng ứng với mỗi ảnh trong bộ dữ liệu. Ở câu hỏi này chúng ta sẽ xây dựng các mô hình phân loại SVM

Khai báo đối tượng SVM:

In [ ]:
svm = sklearn.svm.SVC(kernel="linear", C=0.025)
print(svm)

SVC(C=0.025, kernel='linear')


Huấn luyện, kiểm tra độ chính xác của mô hình tương tự như các bài học trước (sử dụng các hàm fit, predict, score,... của đối tượng svm)

Huấn luyện mô hình:

In [ ]:
svm.fit(X_tr, Y_tr)

Tính độ chính xác trên tập dữ liệu huấn luyện:

In [ ]:
svm.score(X_tr, Y_tr)

Tính độ chính xác trên tập dữ liệu test:

In [ ]:
svm.score(X_te, Y_te)

## Dùng mô hình đã huấn luyện dự đoán hình ảnh thực tế

Cuối cùng ta sẽ dùng thử mô hình đã huấn luyện để đưa ra dự đoán về một hình ảnh thực tế. Chúng ta sẽ làm lần lượt các bước: đọc ảnh, trích xuất đặc trưng BoW của ảnh, dự đoán sử dụng mô hình đã huấn luyện.

__Bước 1__: Đọc ảnh ở đường dẫn image_test/car.png, lưu ảnh vào biến img

In [ ]:
img = None
img = cv.imread('./data/Sc/Sc_213.bmp')
img=cv.cvtColor(img, cv.COLOR_BGR2RGB)
plt.imshow(img)
plt.show()
my_X = [img]

__Bước 2__: Trích xuất đặc trưng SIFT (lưu vào biến my_image_descriptors) và BoW (lưu vào biến my_X_features) từ my_X:

In [ ]:
my_image_descriptors = None
my_X_features = None
my_image_descriptors = extract_sift_features(my_X)
my_X_features = create_features_bow(my_image_descriptors, BoW, num_clusters)
print(len(my_image_descriptors))
print(my_image_descriptors[0].shape)
print(my_X_features[0].shape)
print(my_X_features[0])

__Bước 3:__ Sử dụng mô hình đã huấn luyện để dự đoán, kết quả dự đoán lưu vào biến my_y_pred

In [ ]:
y_pred = None
y_pred = svm.predict(my_X_features)
print(y_pred)
print('Loại lỗi: ' + classes[y_pred[0]])

